In [ ]:
# Install the Ultralytics library
!pip install ultralytics

# Import and check that the GPU is active
import torch
print(f"GPU available: {torch.cuda.is_available()}")
!nvidia-smi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.0 MB/s eta 0:00:00
GPU available: False
/bin/bash: line 1: nvidia-smi: command not found


In [ ]:
print(f"GPU available: {torch.cuda.is_available()}")

GPU available: True


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Qj6PeOKotXZAq4cKGokg")
project = rf.workspace("jeremy-w3v4a").project("parking-detection-jeremykevin")
version = project.version(10)
dataset = version.download("yolov11")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 22.4 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to parking-detection-jeremyKevin-10 in yolov11:: 100%|██████████| 11142/11142 [00:03<00:00, 3242.73it/s]


In [ ]:
from ultralytics import YOLO

# Load the nano model (yolov8n.pt) for object detection
model = YOLO('yolo11n.pt')

# Train the model
# data: path to our data.yaml file
# imgsz: 640 (matches the dataset's preprocessing)
# epochs: 50 (a good starting point)
# batch: 16 (a safe batch size for a T4 GPU)
results = model.train(
    data=f"{dataset.location}/data.yaml",
    epochs=100,
    imgsz=640,
    batch=16,
    name='yolov11n_parking'
)

print("Training complete!")
print(f"Best model saved to: {results.save_dir}/weights/best.pt")

Ultralytics 8.3.222 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/parking-detection-jeremyKevin-10/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov11n_parking, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, 

In [ ]:
from ultralytics import YOLO

# The best model is saved in the 'results.save_dir' path
best_model_path = "best.pt"

# Load our newly trained model
model = YOLO(best_model_path)

# Run validation on the 'test' split
metrics = model.val(split='test')

print("Validation Results:")
print(f"mAP50-95: {metrics.box.map}")  # Overall accuracy
print(f"mAP50: {metrics.box.map50}")  # Accuracy at 50% IOU

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.222 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1600.2±538.4 MB/s, size: 77.7 KB)
val: Scanning /content/parking-detection-jeremyKevin-10/test/labels... 830 images, 1 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 830/830 1.4Kit/s 0.6s
val: New cache created: /content/parking-detection-jeremyKevin-10/test/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 1349, len(boxes) = 21487. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply 

In [ ]:
# 'metrics' is the variable from your Step 4 (model.val())
# 'model' is your loaded model

print("--- Final Model Metrics for Research Paper ---")

# mAP (mean Average Precision)
print(f"mAP@50-95:  {metrics.box.map:.4f}")
print(f"mAP@50:     {metrics.box.map50:.4f}")
print(f"mAP@75:     {metrics.box.map75:.4f}")

# Precision and Recall
print(f"Precision:  {metrics.box.mp:.4f}")
print(f"Recall:     {metrics.box.mr:.4f}")
print("-----------------------------------------------")

# --- THIS IS THE CORRECTED PART ---
# Get metrics for *each* class ('car' and 'free')
# .maps is an array, so we use enumerate to get the class_id (index)
class_map_array = metrics.maps
class_names = model.names

print("\n--- Metrics Per Class ---")
for class_id, map_val in enumerate(class_map_array):
    # Get the class name using the class_id as the index
    class_name = class_names[class_id]
    print(f"Class: {class_name} (id {class_id}) | mAP@50-95: {map_val:.4f}")

--- Final Model Metrics for Research Paper ---
mAP@50-95:  0.6132
mAP@50:     0.9047
mAP@75:     0.6442
Precision:  0.9324
Recall:     0.8618
-----------------------------------------------

--- Metrics Per Class ---
Class: car (id 0) | mAP@50-95: 0.5694
Class: free (id 1) | mAP@50-95: 0.6570


In [ ]:
from google.colab import files

print("Downloading your trained model (best.pt)...")
files.download(best_model_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import random
from IPython.display import Image, display
from ultralytics import YOLO

# --- 1. Get Path to Test Images ---
# 'dataset.location' was set when you downloaded from Roboflow
test_images_path = f"{dataset.location}/test/images"

# --- 2. Pick a Random Image ---
try:
    random_image_name = random.choice(os.listdir(test_images_path))
    random_image_path = os.path.join(test_images_path, random_image_name)
    print(f"Selected random image: {random_image_name}")

    # --- 3. Load Your Trained Model ---
    # 'best_model_path' was set after your training
    model = YOLO(best_model_path)

    # --- 4. Run Prediction ---
    # 'conf=0.5' means it will only show detections it's 50% confident about
    # 'save=True' tells it to save the annotated image to a file
    results = model.predict(random_image_path, save=True, conf=0.5)

    # --- 5. Display the Result ---
    # The path to the saved image is in the results object
    saved_image_path = results[0].save_path

    print(f"Results with bounding boxes saved to: {saved_image_path}")

    # Display the annotated image directly in your notebook
    display(Image(filename=saved_image_path))

except Exception as e:
    print(f"An error occurred: {e}")
    print("Please make sure 'dataset' and 'best_model_path' variables are set.")

Selected random image: img97-Copy_jpg.rf.0a5d94e4e967061b1f42725a80beb035.jpg

image 1/1 /content/parking-detection-jeremyKevin-10/test/images/img97-Copy_jpg.rf.0a5d94e4e967061b1f42725a80beb035.jpg: 640x640 10 cars, 4 frees, 13.9ms
Speed: 4.0ms preprocess, 13.9ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)
Results saved to /content/runs/detect/predict
An error occurred: 'Results' object has no attribute 'save_path'. See valid attributes below.

    A class for storing and manipulating inference results.

    This class provides comprehensive functionality for handling inference results from various
    Ultralytics models, including detection, segmentation, classification, and pose estimation.
    It supports visualization, data export, and various coordinate transformations.

    Attributes:
        orig_img (np.ndarray): The original image as a numpy array.
        orig_shape (tuple[int, int]): Original image shape in (height, width) format.
        boxes (Boxes |

In [ ]:
import shutil
import os

zip_path = "/content/runs.zip"
shutil.make_archive(zip_path.replace(".zip", ""), 'zip', "/content/runs")
print(f"Zipped folder saved to: {zip_path}")

Zipped folder saved to: /content/runs.zip


In [ ]:
from google.colab import files

print(f"Downloading {zip_path}...")
files.download(zip_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
from ultralytics import YOLO

# Assuming your video is in the current directory, replace 'your_video.mp4' with the actual filename
video_path = '/content/parking_video.mp4' # Replace with your video file path
output_video_path = '/content/parking_detection_output.mp4' # Path to save the output video

# Load your trained model
# 'best_model_path' was set after your training
model = YOLO(best_model_path)

# Run prediction on the video
# 'conf=0.5' means it will only show detections it's 50% confident about
# 'save=True' tells it to save the annotated video to a file
results = model.predict(video_path, save=True, conf=0.5, project='/content', name='video_predictions')

# When saving video, the output is saved to a directory, not a single file with save_path
# The save directory is available in results[0].save_dir
print(f"Processed video saved to directory: {results[0].save_dir}")


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/498) /content/parking_video.mp4: 384x640 53 cars, 7 frees, 61.8ms
video 1/1 (frame 2/498) /content/parking_video.mp4: 384x640 53 cars, 7 frees, 13.8ms
video 1/1 (frame 3/498) /content/parking_video.mp4: 384x640 53 cars, 6 frees, 8.5ms
video 1/1 (frame 4/498) /content/parking_video.mp4: 384x640 53 cars, 6 frees, 9.1ms
video 1/1 (frame 5/498) /content/parking_video.mp4: 384x640 53 cars, 5 frees, 11.8ms
video 1/1 (frame 6/498) /content/

In [ ]:
import os
from ultralytics import YOLO

# Assuming your video is in the current directory, replace 'your_video.mp4' with the actual filename
video_path = '/content/parking_video.mp4' # Replace with your video file path
output_video_path = '/content/parking_detection_output.mp4' # Path to save the output video

# Load your trained model
# 'best_model_path' was set after your training
model = YOLO(best_model_path)

# Run prediction on the video
# 'conf=0.5' means it will only show detections it's 50% confident about
# 'save=True' tells it to save the annotated video to a file
results = model.predict(video_path, save=True, conf=0.5, project='/content', name='video_predictions')

# When saving video, the output is saved to a directory, not a single file with save_path
# The save directory is available in results[0].save_dir
print(f"Processed video saved to directory: {results[0].save_dir}")


WARNING ⚠️ 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/498) /content/parking_video.mp4: 384x640 53 cars, 7 frees, 61.8ms
video 1/1 (frame 2/498) /content/parking_video.mp4: 384x640 53 cars, 7 frees, 13.8ms
video 1/1 (frame 3/498) /content/parking_video.mp4: 384x640 53 cars, 6 frees, 8.5ms
video 1/1 (frame 4/498) /content/parking_video.mp4: 384x640 53 cars, 6 frees, 9.1ms
video 1/1 (frame 5/498) /content/parking_video.mp4: 384x640 53 cars, 5 frees, 11.8ms
video 1/1 (frame 6/498) /content/

**Note:** Before running the code above, make sure to upload your video file to the specified `video_path` in the code cell.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')